In [1]:
! pip3 install bayesian_optimization
! pip3 install xgboost

  Using cached bayesian_optimization-1.2.0-py3-none-any.whl


In [8]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore') 

In [2]:
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [3]:
train_values = pd.get_dummies(train_values)

In [6]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100))  

In [7]:
cross_val_score(pipe, train_values, train_labels, scoring='f1_micro', cv=10).mean()

/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[14:49:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[14:54:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:25:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:30:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7439342157662274

In [14]:
train_values1 = pd.read_csv('train_values_short1.csv', index_col='building_id')

In [15]:
con_ic = train_values1[['min_geo_id', "max_geo_id"]].merge(train_values, left_index=True, right_index=True)
con_ic.drop(columns=["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], inplace=True)

In [ ]:
pipe1 = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100))  

In [ ]:
cross_val_score(pipe1, con_ic, train_labels, scoring='f1_micro', cv=10).mean()

/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:34:42] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:39:04] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:09:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:14:00] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7584238004330297

In [4]:
def funcion_a_optimizar(gamma, min_child_weight):
    pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100,
                                  gamma=gamma,  min_child_weight=min_child_weight))  
                       
    return cross_val_score(pipe, con_ic, train_labels, scoring='f1_micro', cv=8).mean()

In [5]:
limites = {'gamma':(0,0.5), 'min_child_weight':(4,10)} 

In [7]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    random_state=645645,
    verbose=5,
    pbounds=limites,
)

load_logs(optimizer, "./logs/XGBoost/baseline_gamma_y_min_child_weihgt.json")
logger = JSONLogger(path="./logs/XGBoost/logs_nuevos_baseline.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [8]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, min=0):
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for resultado in resultados_importantes:
        print("target: " , resultado['target'])
        for key,value in resultado['params'].items():
            print("\t\t", key, ':', value)
        print()

In [18]:
optimizer.probe({'gamma':0, "min_child_weight":6}, lazy=True)

In [ ]:
optimizer.maximize(
    init_points=5,
    n_iter=10,
)

/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:34:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:38:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[20:24:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[20:28:23] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/srv/conda/envs/saturn/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/srv/conda/envs/saturn/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[20:32:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [9]:
mostrar(optimizer.res)

target:  0.7600392960291122
		 gamma : 0.07608257721855993
		 min_child_weight : 8.395360004452915

target:  0.7594138266149825
		 gamma : 0.0
		 min_child_weight : 6.0

target:  0.7593025418945927
		 gamma : 0.4828234431909433
		 min_child_weight : 8.990625220573373

target:  0.7591643993826773
		 gamma : 0.005901023768015878
		 min_child_weight : 7.456488533579913

target:  0.7591298592195809
		 gamma : 0.10590465918062508
		 min_child_weight : 4.2317381210384415

target:  0.7589034627284474
		 gamma : 0.34475265000828925
		 min_child_weight : 5.994121570482571

target:  0.7588535727822073
		 gamma : 0.1547196916974214
		 min_child_weight : 5.004542248426331



Creo que lo mejor seria dejar ese valor de min_child_whigit (que es el que tiene mas impacto) y probar un par de valores mas del gamma

### Pruebo agregar agrupando las superestrtrucutaras por algun material

In [15]:
mud = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_mud_mortar_brick']

rc = ['has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered']

stone = ['has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',  'has_superstructure_cement_mortar_stone']

brick = ['has_superstructure_cement_mortar_brick',  'has_superstructure_mud_mortar_brick']

cement = [ 'has_superstructure_cement_mortar_brick', 'has_superstructure_cement_mortar_stone']

In [16]:
grupos = {
    "mud":mud,
    'rc':rc,
    'stone':stone,
    "cement":cement
}

In [17]:
con_grupos = con_ic.copy(deep=True)
for i in grupos:
    con_grupos[i] = con_ic[grupos[i]].max(axis='columns')

In [18]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=24, max_depth= 10, n_estimators= 100,
                                  gamma=0.07608257721855993,  min_child_weight= 8.395360004452915, verbosity=0)) 

In [15]:
cross_val_score(pipe, con_ic, train_labels, scoring='f1_micro', cv=10).mean()

0.7595251082066359

In [19]:
cross_val_score(pipe, con_grupos, train_labels, scoring='f1_micro', cv=10).mean()

0.7591452178596393

In [ ]:
con_grupos.drop(columns=["rc", "stone", "cement"], inplace=True)

In [25]:
cross_val_score(pipe, con_grupos, train_labels, scoring='f1_micro', cv=10).mean()

0.7590761366206088

Resultado: claramente empeoro todo asi que abandono la idea

# Submission

In [16]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100,
                                  gamma = 0.07608257721855993, min_child_weight = 8.395360004452915))  

In [8]:
pipe.fit(con_ic, train_labels)

[21:05:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=10, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=4, num_parallel_tree=1,
                               objective='multi:softprob', random_state=2021,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [25]:
test_values = pd.read_csv('test_values.csv', index_col='building_id')
test_values1 = pd.read_csv('Archivos auxiliares KNN/test_values_short1.csv', index_col='building_id')
test_values = pd.get_dummies(test_values)
test_values = test_values1[['min_geo_id', "max_geo_id"]].merge(test_values, left_index=True, right_index=True)
test_values.drop(columns=["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], inplace=True)

In [26]:
preds = pipe.predict(test_values)

In [27]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [26]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [27]:
submission.to_csv('subs/xgboost_baseline_002.csv', index=True)

### SIn gamma y min child wheight

In [7]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100)) 

In [28]:
# Corri igual lo de arriba
submission.to_csv('subs/xgboost_baseline_003.csv', index=True)

# Borrar los geo_ids 2 y 3

In [9]:
pipe = make_pipeline(StandardScaler(), 
                     XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 100, verbosity=0))  

In [8]:
cross_val_score(pipe, train_values, train_labels, scoring='f1_micro', cv=10).mean()

0.7439342157662274

In [9]:
train_values.drop(columns=["geo_level_2_id","geo_level_3_id"],inplace=True)

In [13]:
cross_val_score(pipe, train_values, train_labels, scoring='f1_micro', cv=10).mean()

0.6994716043883464